# Requirements

In [ ]:
import os, sys

pkg_path = 'D:\\Projects\\System_Trading\\JnQ\\'

os.chdir(pkg_path)

# mpl_finance_path = 'D:\\python\\python38_1\\projects\\JnQ\\mpl_finance'
# ta_lib_path = 'D:\\python\\python38_1\\projects\\JnQ\\ta_lib'
funcs_path = pkg_path + 'funcs'

if funcs_path not in sys.path:

  try:
    # sys.path.insert(0, '/content/drive/My Drive/Colab Notebooks/JnQ')
    sys.path.insert(0, pkg_path + 'Bank')
    sys.path.insert(0, funcs_path)
    # sys.path.insert(0, mpl_finance_path)
    # sys.path.insert(0, ta_lib_path)
    
  except Exception as e:
    print(e)

In [ ]:
import os
import talib
from funcs.public.idep import *
from funcs.public.plot_check import *
from funcs.public.en_ex_pairing import *
from funcs.public.indicator import *
from funcs.public.broker import *
from funcs.public.ds import *
from ast import literal_eval
import logging
import importlib


import matplotlib.pyplot as plt
from matplotlib import gridspec

# import torch

import numpy as np
import pandas as pd
import scipy.stats as stats
# from sklearn.metrics.pairwise import cosine_similarity

# import bz2
import pickle
# import _pickle as cPickle
import shutil
import json
from easydict import EasyDict
import copy

import datetime
from datetime import datetime
import random
import time
# import warnings

from IPython.display import clear_output
# warnings.simplefilter("ignore", category=RuntimeWarning)

np.seterr(invalid="ignore")
np.set_printoptions(suppress=True)
np.set_printoptions(linewidth=2000) 

pd.set_option('mode.chained_assignment',  None)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Telegramer

In [ ]:
import telegram
from telegram.ext import Updater
from telegram.ext import MessageHandler, Filters

In [ ]:
def echo(self, update, context):
    self.user_text = update.message.text

In [ ]:
#       i. Telegram logger
#               1. chat_id 는 env 동일.
token = "6717940201:AAFc62YReeED3mJKw5drHnT7jp6dzXy0rPE"
msg_bot = telegram.Bot(token=token)
chat_id = "5320962614"

#       ii. Telegram messenger
#           1. init.
user_text = None

updater = Updater(token=token, use_context=True)
dispatcher = updater.dispatcher

echo_handler = MessageHandler(Filters.text & (~Filters.command), echo)
dispatcher.add_handler(echo_handler)

#           2. polling for messenger.
updater.start_polling()

In [ ]:
msg = "ping telegramer"

try:
    msg_bot.sendMessage(chat_id=chat_id, text=msg)
except Exception as e:
    print("error in msg_bot : {}".format(e))

# Get data

## Upbit

In [ ]:
from funcs.upbit.concat_candlestick_ftr_v2 import concat_candlestick
from pyupbit import get_tickers

"""
1. upbit 에서 당일 기준 이전 데이터를 조회할지라도 None 은 성립하지 않는다. --> end_date = today 로 변경 요구.
"""


days = 1  # 330 3

end_date = None  # None 2023-07-04 # "2023-01-06" "2021-04-12" "2021-03-23"
    
intervals = [ 'D']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old
intervals = ['30m', '1h', '2h']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old
intervals = [ '4h',  '1d']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old

time_schedule = {'4h': [21, 17, 13, 9, 5, 1], '1d': [9]}
    
show_process = False
show_process = True


# hour_prev = None    
symbol_list = get_tickers()

while 1:

    time.sleep(1)  # term for small fan sound.
    
    target_symbol_total = []
    for interval in intervals:
        
        timestamp = datetime.now().timestamp()
        hour_current = datetime.fromtimestamp(datetime.now().timestamp()).hour
        hour_prev = datetime.fromtimestamp(datetime.now().timestamp() - 10).hour  # 10 seconds before.
        
        # print("hour_current :", hour_current)
        # print("hour_prev :", hour_prev)
        # print("time_schedule[interval] :", time_schedule[interval])
        
        if not ((hour_current in time_schedule[interval]) and hour_current != hour_prev):
            continue

        target_symbol = []
        for s_i, symbol in enumerate(symbol_list):

            print(s_i)

            if 'KRW' not in symbol:
                continue

            try:
                concated_df, end_date = concat_candlestick(symbol, 
                                                           interval, 
                                                           days, 
                                                           limit=limit_by_itv(interval),
                                                           end_date=end_date, 
                                                           show_process=show_process, 
                                                           timesleep=0.1)


                concated_df2 = fisher_v2(concated_df, 30, itv=interval)    
                # display(concated_df2.tail())
                # break

                target_data = concated_df2.iloc[-2:, -1].to_numpy()

                """
                condition
                """
                # 1. fisher band
                fisher_lower = -1.5
                if target_data[0] < fisher_lower < target_data[1]:
                    target_symbol.append(symbol)

                clear_output(wait=True)          



                # break
                # save_path = os.path.join(save_dir, save_name)
                # concated_df.reset_index().to_feather(save_path, compression='lz4')
                # print(save_path, "saved.\n")
            except Exception as e:
                print("error in save to_feather :", e)
                continue

        target_symbol_total.append([s_.replace('KRW-', '') for s_ in target_symbol])
        print("target_symbol_total :", target_symbol_total)
        
        msg = "target_symbol_total : {} {}".format(interval, target_symbol_total)
        try:
            msg_bot.sendMessage(chat_id=chat_id, text=msg)
        except Exception as e:
            print("error in msg_bot : {}".format(e))
        # break

In [74]:

from funcs.public.broker import itv_binance_to_upbit, limit_by_itv

import pyupbit


def concat_candlestick(symbol, interval, days, limit=candle_limit, end_date=None, timesleep=None, show_process=False):
    # assert limit <= candle_limit, "assert limit < candle_limit"

    """ 
    1. real_trade 에서 자정 지나면 data 부족해지는 문제로 assert, days >= 2 적용
        => imit under 1500 으로 cover 가능 (startTime = None 이면, 자정 이전 data load 함)
            = (startTime 을 자정으로 설정해서 자정 이전의 data 를 가져오기 못한 것)
    """

    if end_date is None:
        end_date = str(datetime.now()).split(' ')[0]
    startTime_ = datetime.timestamp(pd.to_datetime('{} 00:00:00'.format(end_date))) * 1000

    # if interval != '1m':
    if interval == '1d':
        startTime_ -= a_day

    endTime = datetime.timestamp(pd.to_datetime('{} 23:59:59'.format(end_date))) * 1000

    if show_process:
        print(symbol)

    # if days > 1:  # 1일 이상의 data 가 필요한 경우 limit 없이 모두 가져옴
    #     limit = candle_limit

    # df_records = []
    # for day_cnt in range(days):
    day_cnt = 0
    while 1:

        # 1. days = None --> 모든 data 추출.
        if days is not None:
            if day_cnt >= days:
                break

        # a. loop 통해서 하루 이전 데이터를 지속적으로 가져옴
        if day_cnt != 0:
            startTime_ -= a_day
            endTime -= a_day

        try:
            # b.  limit < max_limit, startTime != None 일 경우, last_index 이상하게 (-> 어떻게..?) 나옴 ------ #
            # if limit != candle_limit:
            #     startTime = None  # 변수명 get_candlestick_data 함수 param 이랑 일치시키기 위해 startTime.
            # else:
            #     startTime = int(startTime_)

            df = pyupbit.get_ohlcv("{}".format(symbol),
                                   interval=itv_binance_to_upbit(interval),
                                   count=limit,
                                   period=timesleep,
                                   to=datetime.fromtimestamp(endTime / 1000))
            print("len(df) :", len(df))

            #     i. if count <= 200, timesleep not adj to get_ohlcv()
            if limit <= 200:
                # print("limit :", limit)
                time.sleep(timesleep)

            # c. validation
            #       i. 더이상 추출할 데이터가 없는 경우 break.
            if df is None:
                print("no more data.")
                break

            if show_process:
                print(df.index[0], end=" ~ ")
                print(df.index[-1])
            assert len(df) != 0, "len(df) == 0"

            if day_cnt == 0:
                sum_df = df
            else:
                sum_df = pd.concat([df, sum_df])  # <-- -a_day 이기 때문에 sum_df 와 df 의 위치가 좌측과 같음.

        except Exception as e:
            print('error in get_candlestick_data :', e)
            break
        else:
            day_cnt += 1

    # keep = 'last' 로 해야 중복기준 최신 df 를 넣는건데, 왜 first 로 해놓은건지.
    return sum_df[~sum_df.index.duplicated(keep='last')], end_date


In [ ]:
interval = '1d'
days = 365 * 5
concated_df, end_date = concat_candlestick(symbol_list[0], 
                                           interval, 
                                           days, 
                                           limit=20000,
                                           end_date=end_date, 
                                           show_process=show_process, 
                                           timesleep=0.1)

In [ ]:
symbol = symbol_list[0]
count = int((days * 60 * 24) / itv_to_number(interval))
timesleep = 0.1
                     
df = pyupbit.get_ohlcv("{}".format(symbol),
                                   interval=itv_binance_to_upbit(interval),
                                   count=count,
                                   period=timesleep,
                                   to=end_date)

df

In [111]:
# len(df)
end_date

'2024-01-07'

In [ ]:

end_timestamp = int(datetime.timestamp(pd.to_datetime('{} 23:59:59'.format(end_date))) * 1000)
# start_timestamp = int(datetime.timestamp(pd.to_datetime('{} 00:00:00'.format("2023-12-01"))) * 1000)
days_timestamp = days * 86399000
start_timestamp = end_timestamp - days_timestamp

timestamp_unit = (itv_to_number(interval) * 60000 * limit)

In [59]:
df_res = pd.DataFrame(index=intervals, data=target_symbol_total)
df_res

""
4h
1d


In [ ]:
df_res.to_excel("signal/upbit_res.xlsx")

## Binance_Futures

In [ ]:
# with open(r"D:\Projects\System_Trading\JnQ\Bank\tickers\binance_20240102.pkl", 'wb') as f:
#     pickle.dump(symbol_list, f)

In [ ]:
from binance.um_futures import UMFutures
from funcs.binance.futures_concat_candlestick_ftr_v2 import *
# break


days = 5  # 330 3

end_date = None  # None 2023-07-04 # "2023-01-06" "2021-04-12" "2021-03-23"
    
intervals = ['4h', '1d']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old
intervals = ['15m', '30m', '1h', '4h', '1d']  # ['1m', '3m', '5m', '15m', '30m', '1h', '4h'] - old

limit = 1500
    
show_process = False
show_process = True


 
symbol_list = [s_['symbol'] for s_ in um_futures_client.exchange_info()['symbols']]



target_symbol_total = []
for interval in intervals:
    
    target_symbol = []
    for s_i, symbol in enumerate(symbol_list):
        
        print(s_i)
        
        # if 'KRW' not in symbol:
        #     continue
        
        try:
            concated_df, end_date = concat_candlestick_v2(symbol, 
                                                          interval, 
                                                          days,
                                                          limit=limit,
                                                          end_date=None,
                                                          show_process=True,
                                                          timesleep=0.4)
            
            
            concated_df2 = fisher_v2(concated_df, 30, itv=interval.upper())      
            # display(concated_df2.tail())
            # break
            
            target_data = concated_df2.iloc[-2:, -1].to_numpy()
            
            """
            condition
            """
            # 1. fisher band
            fisher_lower = -1.5
            if target_data[0] < fisher_lower < target_data[1]:
                target_symbol.append(symbol)
                
            clear_output(wait=True)          
            
            
            
            # break
            # save_path = os.path.join(save_dir, save_name)
            # concated_df.reset_index().to_feather(save_path, compression='lz4')
            # print(save_path, "saved.\n")
        except Exception as e:
            print("error in save to_feather :", e)
            continue
     
    target_symbol_total.append([s_ + '.P' for s_ in target_symbol])
    # print("target_symbol_total :", target_symbol_total)
    # break

In [ ]:
df_res = pd.DataFrame(index=intervals, data=target_symbol_total)
df_res

In [ ]:
df_res.to_excel("signal/binance_futures_res.xlsx")